<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/time_standarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Imports

In [ ]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#Manejador de todo

In [ ]:


country = 'br master db'
dir = f'{country}/parquet'
databases = os.listdir(dir)
n_databases = len(databases)

#initialize
df = [None]*n_databases
largest_len_db = 0
n = 0

for file_name in databases:
    db = f'{dir}/{file_name}'
    dataframe = pd.read_parquet(db)
    df[n] = db
    if len(file_name) > largest_len_db:
        largest_len_db = len(file_name)
    n = n + 1

#Print tools
centered_db = centered_len(largest_len_db)
centered_header = centered_db('masterdbs')
dashes = '-'*largest_len_db
print(f' {centered_header} | Edition')
print(f' {dashes} | ------ ')

for d in range(len(df)):
    #file name extract
    dir_len = len(dir)+1
    parquet_len = len('.parquet')
    file_name_len = len(df[d])-parquet_len
    file_name = df[d][dir_len:file_name_len]
    #File name print to log
    print(f' {centered_db(file_name)}', end=' | ')

    #--------Edition--------
    #Load Dataframe
    edition = pd.read_parquet(df[d])
    #Ranking
    try:
        edition['Rank'] = edition['Rank'].str.extract(r'(\d+)')
        #Fechas
        edition['time'] = pd.to_datetime(edition['time'])
        edition['time'] = edition['time'] - timedelta(hours=5)

        #--------Upadting--------
        #File names
        csv_file = f'{country}/csv/{file_name}.csv'
        excel_file = f'{country}/excel/{file_name}.xlsx'
        parquet_file = f'{country}/parquet/{file_name}.parquet'
        #File updating
        edition.to_csv(csv_file,sep="|")
        edition.to_excel(excel_file, index=False)
        edition.to_parquet(parquet_file, allow_truncated_timestamps=True)

        print('Done')
    except AttributeError:
        csv_file = f'{country}/csv/{file_name}.csv'
        excel_file = f'{country}/excel/{file_name}.xlsx'
        parquet_file = f'{country}/parquet/{file_name}.parquet'

        os.remove(csv_file)
        os.remove(excel_file)
        os.remove(parquet_file)

        print('Deleted. File Error: No data in')

#Labs